In [2]:
# check the datasets
from unzip_file import unzip_file

file_path = 'datas/playground-series-s4e4.zip'
output_dir = 'datas/abalone-dataset'
unzip_file(file_path, output_dir)

Unzip to :
datas/abalone-dataset/test.csv
datas/abalone-dataset/train.csv
datas/abalone-dataset/sample_submission.csv
